In [ ]:
import requests
import pandas as pd
import utils.misc as misc
from config import *
import yahoo_fin.stock_info as si
from yahooquery import Ticker
import quandl
import os
from datetime import datetime, timedelta
from finvizfinance.screener.custom import Custom
import time

In [ ]:
BASE_URL = r"https://www.alphavantage.co/query?"
av_api = 'CQLP7J9VZG14Y7O4'
av_api_basil = 'IHLNUQ0G66C0MJID'

using proxies

In [ ]:
def av_api(func, api_key, symbol):
    assert symbol is not None
    symbol = symbol.strip().upper()

    url = f"{BASE_URL}function={func}&symbol={symbol}&apikey={api_key}"
    response = requests.get(url)
    if func == 'EARNINGS':
        try:
            output = pd.DataFrame(response.json()['quarterlyEarnings'])
        except:
            print(response.json())
            output = None
    elif func == 'BALANCE_SHEET':
        print(response)
        print(response.json())
        return pd.DataFrame(response.json()['quarterlyReports'])
    elif func == 'INCOME_STATEMENT':
        return pd.DataFrame(response.json()['quarterlyReports'])
    return output

In [ ]:
def get_av_data(symbol):
    #For reported Earnings & Analyst Estimated EPS, totalRevenue, commonStockSharesOutstanding
    df1 = av_api('EARNINGS',av_api, symbol)
    print(df1)
    df2 = av_api('INCOME_STATEMENT',av_api, symbol)[['fiscalDateEnding','totalRevenue']]
    print(df2)
    df1 = df1.merge(df2, on='fiscalDateEnding', how='left')
    df3 = av_api('BALANCE_SHEET',av_api, symbol)[['fiscalDateEnding','commonStockSharesOutstanding']]
    df1 = df1.merge(df3, on='fiscalDateEnding', how='left')

    return df1

df = get_av_data("EXFY")

In [ ]:
df

In [ ]:
# %%timeit

success = 0
for i in range(50):
    while True:
        try:
            df1 = av_api('EARNINGS',av_api_basil, 'META')
            print(df1.head(1))
            success+=1
            break
        except:
            time.sleep(0.001)
            continue

In [ ]:
success

In [ ]:
def get_ex_dates(symbol):
    symbol = 'META'
    tkr = Ticker(symbol)
    df = tkr.sec_filings
    df = df[(df['type'] == '10-Q') | ((df['type'] == '10-K'))]
    df.reset_index(inplace=True)
    df = df[['date','title']]
    df.rename(columns={'date': 'Ex-Date'}, inplace=True)
    return df

In [ ]:
av_api('BALANCE_SHEET', av_api, symbol)

In [ ]:
import yfinance as yf
from yahoofinancials import YahooFinancials

In [ ]:
yahoo_financials = YahooFinancials('AAPL')
data = yahoo_financials.get_historical_price_data(start_date='2019-01-01', 
                                                  end_date='2019-12-31', 
                                                  time_interval='weekly')
aapl_df = pd.DataFrame(data['AAPL']['prices'])
aapl_df = aapl_df.drop('date', axis=1).set_index('formatted_date')
aapl_df.head()

In [ ]:
from __future__ import print_function
import time
import intrinio_sdk as intrinio
from intrinio_sdk.rest import ApiException

intrinio.ApiClient().set_api_key('YOUR_API_KEY')
intrinio.ApiClient().allow_retries(True)

identifier = 'AAPL'
page_size = 100
next_page = ''

response = intrinio.ZacksApi().get_zacks_analyst_ratings(identifier=identifier, page_size=page_size, next_page=next_page)
print(response)
    

##IGNORE FOR NOW

    NASDAQ DATALINK - FOR SHORT INTEREST

In [ ]:
ndl_api = 'dys7Lqv_HziWD1ZFk4ry'

In [ ]:
quandl.ApiConfig.api_key = ndl_api

In [ ]:
os.chdir('/Users/yash/Desktop/Trading/intern/Basil/finra_short_interest')

In [ ]:
df2 = pd.read_csv('equityshortinterest_2023_02_26_05_44_02.csv')

Q4 Earnings 2022

In [ ]:
df1.to_csv('us_stocks.csv')

In [ ]:
earnings_df = pd.DataFrame(columns=['Symbol', 'Ex-Date', 'title'])

In [ ]:
tickers = df1['Ticker']
for t in tickers:
    dft = get_ex_dates(t)
    dft["Date"] = dft["Ex-Date"].apply(
        lambda x: datetime.strptime(x, "%Y-%m-%d")
    )
    dft = dft[dft['Date'] >= datetime(2023,1,1)]
    if dft.shape[0] > 0:
        earnings_df.loc[len(earnings_df.index)] = [t]+list(dft.iloc[0,1:]) 

In [ ]:
import csv
import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
CSV_URL = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=IBM&interval=1min&slice=year1month1&apikey={av_api}'

with requests.Session() as s:
    download = s.get(CSV_URL)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)
    for row in my_list:
        print(row)

In [ ]:
import requests
from bs4 import BeautifulSoup

headers = {
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:105.0) Gecko/20100101 Firefox/105.0"
}
url = f"https://finance.yahoo.com/quote/MSFT/analysis?p=MSFT"
soup = BeautifulSoup(requests.get(url, headers=headers).content, "html5lib")

for table in soup.select("table"):
    th_row = [th.text for th in table.find_all("th")]
    print(th_row)

    for tr in table.select("tr:has(td)"):
        td_row = [td.text for td in tr.find_all("td")]
        print(td_row)

    print()

In [ ]:
df2,e = ek.get_data('GOOGL.O',['TR.RevenueActValue.date','TR.RevenueActValue','TR.EPSActValue'],parameters = {'SDate':'0','EDate':'-5','Period':'FQ0','Frq':'FQ'}) 